# Create index
The index file is a dataframe that contains all metadata to all wells in the LINCS project. While other index files have existed, I recreated this to be sure about the correctness.

In other notebooks I will be using this data to annotate the profiles and aggregated data from DP.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [23]:
# the barcode is found here:
# https://github.com/broadinstitute/lincs-cell-painting/blob/master/metadata/platemaps/2016_04_01_a549_48hr_batch1/barcode_platemap.csv

barcode = pd.read_csv('barcode_platemap.csv')
print(barcode.shape)
barcode.head()

(140, 4)


,Assay_Plate_Barcode,Plate_Map_Name,Batch_Number,Batch_Date
0,SQ00015201,C-7161-01-LM6-017,1,2016-03-22
1,SQ00015202,C-7161-01-LM6-018,1,2016-03-22
2,SQ00015200,C-7161-01-LM6-016,1,2016-03-22
3,SQ00015204,C-7161-01-LM6-020,1,2016-03-22
4,SQ00015205,C-7161-01-LM6-021,1,2016-03-22


In [24]:
# Map of Broad ID to Sample to MOA
# https://github.com/broadinstitute/lincs-cell-painting/blob/master/metadata/moa/repurposing_info_external_moa_map_resolved.tsv

moa_map = pd.read_csv('repurposing_info_external_moa_map_resolved.tsv', sep='\t')
print(moa_map.shape)
moa_map.head()

(1570, 10)


,broad_sample,broad_id,pert_iname,InChIKey14,moa,target,broad_date,clinical_phase,alternative_moa,alternative_target
0,BRD-A00147595-001-01-5,BRD-A00147595,balaglitazone,IETKPTYAGKZLKY,insulin sensitizer|PPAR receptor partial agonist,PPARG,broad_id_20170327,Phase 3,NaN,NaN
1,BRD-A00218260-001-03-4,BRD-A00218260,flutrimazole,QHMWCHQXCUNUAK,sterol demethylase inhibitor,NaN,broad_id_20170327,Launched,NaN,NaN
2,BRD-A00376169-001-01-6,BRD-A00376169,KBG,FOLRUCXBTYDAQK,neprilysin inhibitor,MME,broad_id_20170327,Phase 2,NaN,NaN
3,BRD-A00546892-001-02-6,BRD-A00546892,biperiden,YSXKPIUOCJLQIE,acetylcholine receptor antagonist,CHRM1|CHRM2|CHRM3|CHRM4|CHRM5,broad_id_20170327,Launched,NaN,NaN
4,BRD-A00938334-001-01-3,BRD-A00938334,drospirenone,METQSPRSQINEEU,mineralocorticoid receptor antagonist,AR|NR3C2|PGR,broad_id_20170327,Launched,NaN,NaN


In [25]:
# Go into the lincs directory and pull the Broad ID from every plate.
lincs_dir = '/Users/mbornhol/git/lincs-cell-painting/'

pd_ls =[]
for map_name in barcode.Plate_Map_Name.unique():
    sub_barcode = barcode.query("Plate_Map_Name == @map_name")
    map = pd.read_csv(os.path.join(lincs_dir, 'metadata/platemaps/2016_04_01_a549_48hr_batch1/platemap/{}.txt'.format(map_name)), sep='\t')
    merged = pd.merge(sub_barcode, map, how='left', left_on='Plate_Map_Name', right_on='plate_map_name')
    pd_ls.append(merged)


In [26]:
# At this point we have all 140 plates with 384 wells = 53.760
df = pd.concat(pd_ls)
print(df.shape)
df.head()

(53760, 10)


,Assay_Plate_Barcode,Plate_Map_Name,Batch_Number,Batch_Date,plate_map_name,well_position,broad_sample,mg_per_ml,mmoles_per_liter,solvent
0,SQ00015201,C-7161-01-LM6-017,1,2016-03-22,C-7161-01-LM6-017,A01,NaN,NaN,NaN,DMSO
1,SQ00015201,C-7161-01-LM6-017,1,2016-03-22,C-7161-01-LM6-017,A02,NaN,NaN,NaN,DMSO
2,SQ00015201,C-7161-01-LM6-017,1,2016-03-22,C-7161-01-LM6-017,A03,NaN,NaN,NaN,DMSO
3,SQ00015201,C-7161-01-LM6-017,1,2016-03-22,C-7161-01-LM6-017,A04,NaN,NaN,NaN,DMSO
4,SQ00015201,C-7161-01-LM6-017,1,2016-03-22,C-7161-01-LM6-017,A05,NaN,NaN,NaN,DMSO


The next step is now to add a row for the sites and to fill every well with 9 sites.

In [27]:
sites = pd.DataFrame(range(1,10), columns=['Site'])
df = pd.merge(df, sites, how='cross')

In [28]:
print(df.shape)
df.head()

(483840, 11)


,Assay_Plate_Barcode,Plate_Map_Name,Batch_Number,Batch_Date,plate_map_name,well_position,broad_sample,mg_per_ml,mmoles_per_liter,solvent,Site
0,SQ00015201,C-7161-01-LM6-017,1,2016-03-22,C-7161-01-LM6-017,A01,NaN,NaN,NaN,DMSO,1
1,SQ00015201,C-7161-01-LM6-017,1,2016-03-22,C-7161-01-LM6-017,A01,NaN,NaN,NaN,DMSO,2
2,SQ00015201,C-7161-01-LM6-017,1,2016-03-22,C-7161-01-LM6-017,A01,NaN,NaN,NaN,DMSO,3
3,SQ00015201,C-7161-01-LM6-017,1,2016-03-22,C-7161-01-LM6-017,A01,NaN,NaN,NaN,DMSO,4
4,SQ00015201,C-7161-01-LM6-017,1,2016-03-22,C-7161-01-LM6-017,A01,NaN,NaN,NaN,DMSO,5


In [29]:
# Merge Moa data via the sample id.
df = pd.merge(df, moa_map, how='left', on='broad_sample')

---
From now onward, we only rename the columns and delete some unimportant ones. The structure of the df is not changed anymore.

In [30]:
# only keep important columns
use_cols = ['Assay_Plate_Barcode', 'well_position', 'Site', 'broad_sample', 'moa', 'mmoles_per_liter', 'pert_iname',  'target', 'Plate_Map_Name', 'Batch_Number']
df = df[use_cols]
df.rename(columns = {'Assay_Plate_Barcode': 'Metadata_Plate', 'well_position': 'Metadata_Well', 'broad_sample': 'Metadata_Broad_Sample', 'moa' : 'Metadata_Moa', 'Site' : 'Metadata_Site'}, inplace = True)
df.head(3)

,Metadata_Plate,Metadata_Well,Metadata_Site,Metadata_Broad_Sample,Metadata_Moa,mmoles_per_liter,pert_iname,target,Plate_Map_Name,Batch_Number
0,SQ00015201,A01,1,NaN,NaN,NaN,NaN,NaN,C-7161-01-LM6-017,1
1,SQ00015201,A01,2,NaN,NaN,NaN,NaN,NaN,C-7161-01-LM6-017,1
2,SQ00015201,A01,3,NaN,NaN,NaN,NaN,NaN,C-7161-01-LM6-017,1


In [31]:
# We replace the NaN values with DMSO and unknowns
df.Metadata_Broad_Sample.replace(np.nan, 'DMSO', regex=True, inplace=True)
df.Metadata_Moa.replace(np.nan, 'NA', regex=True, inplace=True)
df.mmoles_per_liter.replace(np.nan, 'NA', regex=True, inplace=True)
df.pert_iname.replace(np.nan, 'NA', regex=True, inplace=True)
df.target.replace(np.nan, 'NA', regex=True, inplace=True)

In [32]:
# sort the df by plate and Well and site
df.sort_values(by=['Metadata_Plate', 'Metadata_Well', 'Metadata_Site'], inplace = True, ignore_index=True)

In [33]:
# show what the non DMSO data looks like:
df.tail()

,Metadata_Plate,Metadata_Well,Metadata_Site,Metadata_Broad_Sample,Metadata_Moa,mmoles_per_liter,pert_iname,target,Plate_Map_Name,Batch_Number
483835,SQ00015233,P24,5,BRD-K41731458-001-15-1,antibacterial agent,0.041152,triclosan,DNMT1,C-7161-01-LM6-014,3
483836,SQ00015233,P24,6,BRD-K41731458-001-15-1,antibacterial agent,0.041152,triclosan,DNMT1,C-7161-01-LM6-014,3
483837,SQ00015233,P24,7,BRD-K41731458-001-15-1,antibacterial agent,0.041152,triclosan,DNMT1,C-7161-01-LM6-014,3
483838,SQ00015233,P24,8,BRD-K41731458-001-15-1,antibacterial agent,0.041152,triclosan,DNMT1,C-7161-01-LM6-014,3
483839,SQ00015233,P24,9,BRD-K41731458-001-15-1,antibacterial agent,0.041152,triclosan,DNMT1,C-7161-01-LM6-014,3


In [34]:
# Save index file.
df.to_csv('full_index.csv',index=False)

In [39]:
# Index with only Plate and Well
plate_well = df.drop_duplicates(subset=['Metadata_Plate', 'Metadata_Well'], keep='first', ignore_index=True)
plate_well.drop(columns='Metadata_Site', inplace=True)

/Users/mbornhol/miniconda3/envs/noncyto/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [40]:
# save
plate_well.to_csv('full_well_index.csv', index=False)

In [ ]:
TODO
Compare to the actual images on S3!